In [4]:
import datetime
from datetime import date
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.session import SparkSession


from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType

## Obtain last_idx_peterman_line

In [3]:
'''
use eandev; 
drop table if exists eandev.yixli_last_idx_peterman_line;
create table eandev.yixli_last_idx_peterman_line as    
select  
  a.message_id,
  IF(b.message_id is null, 0, b.last_idx_peterman_line) as last_idx_peterman_line,
  IF(c.message_id is null, 0, c.last_hotel_result_index) as last_hotel_result_index
from eandev.HotelMessages as a 
left join (
  select  
   message_id
  ,MAX(v1.hotel_result_index) as last_idx_peterman_line
  from eandev.HotelMessages 
  lateral view explode(hotels) hotel_tbl as v1
  where load_date between '20200301' and '20200401'
  and ((v1.property_level_neg = true) or (v1.chain_level_neg = true) or (v1.preferred = true))
  and message_id is not null 
  and message_id <> ''
  and ((original_message_id is null) or (original_message_id == '')) 
  and type = 'HSR'  
  group by message_id
) as b ON a.message_id = b.message_id
left join (
  select  
   message_id
  ,MAX(v1.hotel_result_index) as last_hotel_result_index
  from eandev.HotelMessages 
  lateral view explode(hotels) hotel_tbl as v1
  where load_date between '20200301' and '20200401'
  and message_id is not null 
  and message_id <> ''
  and ((original_message_id is null) or (original_message_id == '')) 
  and type = 'HSR'  
  group by message_id
) as c ON a.message_id = c.message_id
where a.load_date between '20200301' and '20200401'
and a.message_id is not null 
and a.message_id <> ''
and ((original_message_id is null) or (original_message_id == '')) 
and a.type = 'HSR'
'''

"\nuse eandev; \ndrop table if exists eandev.yixli_last_idx_peterman_line;\ncreate table eandev.yixli_last_idx_peterman_line as    \nselect  \n  a.message_id,\n  IF(b.message_id is null, 0, b.last_idx_peterman_line) as last_idx_peterman_line,\n  IF(c.message_id is null, 0, c.last_hotel_result_index) as last_hotel_result_index\nfrom eandev.HotelMessages as a \nleft join (\n  select  \n   message_id\n  ,MAX(v1.hotel_result_index) as last_idx_peterman_line\n  from eandev.HotelMessages \n  lateral view explode(hotels) hotel_tbl as v1\n  where load_date between '20191201' and '20200101'\n  and ((v1.property_level_neg = true) or (v1.chain_level_neg = true) or (v1.preferred = true))\n  and message_id is not null \n  and message_id <> ''\n  and ((original_message_id is null) or (original_message_id == '')) \n  and type = 'HSR'  \n  group by message_id\n) as b ON a.message_id = b.message_id\nleft join (\n  select  \n   message_id\n  ,MAX(v1.hotel_result_index) as last_hotel_result_index\n  from

In [3]:
peterman_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("last_idx_peterman_line", T.IntegerType(), True),     #2 
    T.StructField("last_hotel_result_index", T.IntegerType(), True)     #3
])

In [4]:
file_loc = "s3://ege-ds-workshops-corp/yixli/last_idx_peterman_line_2019.csv"

peterman_df = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(peterman_schema).load(file_loc)

# load_date between '20191201' and '20200101'

print(peterman_df.count())

1085193

In [23]:
peterman_df.show(50)

+--------------------+----------------------+-----------------------+
|          message_id|last_idx_peterman_line|last_hotel_result_index|
+--------------------+----------------------+-----------------------+
|01321a12-beab-468...|                     3|                     29|
|01573036-848e-48c...|                     0|                      8|
|016fabaa-be7f-473...|                     0|                     30|
|01812298-95a5-47e...|                     3|                     29|
|01884eea-70fb-487...|                     4|                     29|
|01c5055a-14a0-43e...|                     0|                     29|
|01e27c58-328c-438...|                     0|                      9|
|01efeab7-5a3f-489...|                     0|                      0|
|02038b64-6011-46c...|                    29|                     29|
|0221cddc-ac25-4cd...|                     0|                     29|
|027be122-64f0-429...|                     0|                      0|
|0291c6a0-7a77-400..

## Search data

In [ ]:
'''
use eandev;

drop table if exists eandev.yixli_search_commission_test;
create table eandev.yixli_search_commission_test as   
select 
message_id
,message_timestamp as message_date
,check_in_date
,check_out_date
,tuid
,v1.hotel_id
,v1.hotel_result_index
,v2.rate_index
,v2.price.amount as rate_amount
,v2.price.currency as rate_currency
,(case when v2.rate_type = 'GDS_CHAIN_NEGOTIATED' then 'NEG'                             
  when v2.rate_type = 'GDS_CONSORTIA' then 'GDS'
  when v2.rate_type = 'GDS_NEGOTIATED' then 'NEG'
  when v2.rate_type = 'GDS_PUBLISHED' then 'GDS'                             
  when v2.rate_type = 'EPR_MERCHANT' then 'EPRM'
  when v2.rate_type = 'ESR_DIRECT_AGENCY' then 'ESRA'
  when v2.rate_type = 'EPR_DIRECT_AGENCY' then 'EPRA'
  when v2.rate_type = 'ESR_MERCHANT' then 'ESRM'
  else v2.rate_type end) as rate_type
,NVL(v3.commission_base, 0.0) as commission_base
,NVL(v3.commission_tax, 0.0) as commission_tax
,NVL(v3.currency, 'USD') as currency
from eandev.HotelMessages
lateral view explode(travelers) travl_tbl as tuid
lateral view explode(hotels) hotel_tbl as v1
lateral view explode(v1.prices) rate_tbl as v2
lateral view outer explode(v2.commission_list) cms_tbl as v3 
where load_date between '20191001' and '20200101'
and message_id is not null 
and message_id <> ''
and v2.selected = True
and type = 'HSR'


use eandev;
drop table if exists eandev.yixli_src_amentity_test;
create table eandev.yixli_src_amentity_test as
select
 a.message_id
,a.message_date
,a.check_in_date
,a.check_out_date
,a.tuid
,a.hotel_id
,a.hotel_result_index
,a.score
,a.city
,a.star_rating
,a.filter_want_in_policy_rates_only
,array_contains(a.filter_rate_amenities,"EligibleForLoyalty") as  filter_eligible_for_loyalty
,array_contains(a.filter_rate_amenities,"FreeBreakfast") as  filter_free_breakfast
,array_contains(a.filter_rate_amenities,"FreeWifi") as  filter_free_wifi
,array_contains(a.filter_rate_amenities,"FreeParking") as  filter_free_parking
,concat_ws("|",collect_list(cast(a.rate_index as string))) as rate_index
,concat_ws("|",collect_list(cast(a.rate_amount as string))) as rate_amount
,concat_ws("|",collect_list(cast(a.rate_currency as string))) as rate_currency
,concat_ws("|",collect_list(a.rate_type)) as rate_type
,concat_ws("|",collect_list(cast(a.hot_mip_rate) as string) as hot_mip_rate
,concat_ws("|",collect_list(cast(array_contains(a.amnts,"EligibleForLoyalty") as string))) as eligible_for_loyalty
,concat_ws("|",collect_list(cast(array_contains(a.amnts,"FreeBreakfast") as string))) as free_breakfast
,concat_ws("|",collect_list(cast(array_contains(a.amnts,"FreeWifi") as string))) as free_wifi
,concat_ws("|",collect_list(cast(array_contains(a.amnts,"FreeParking") as string))) as free_parking
,concat_ws("|",collect_list(cast(array_contains(a.amnts,"Refundable") as string))) as refundable
from (
select
 message_id
,message_timestamp as message_date
,check_in_date
,check_out_date
,filter_rate_amenities
,filter_want_in_policy_rates_only
,tuid
,v1.hotel_id
,v1.hotel_result_index as hotel_result_index
,v1.sort_model_scores[0].main_score as score
,CONCAT(lower(v1.city), '-',lower(v1.country_code))  as city
,v1.star_rating
,v2.rate_index
,v2.price.amount as rate_amount
,v2.price.currency as rate_currency
,(case when v2.rate_type = 'GDS_CHAIN_NEGOTIATED' then 'NEG'
  when v2.rate_type = 'GDS_CONSORTIA' then 'GDS'
  when v2.rate_type = 'GDS_NEGOTIATED' then 'NEG'
  when v2.rate_type = 'GDS_PUBLISHED' then 'GDS'
  when v2.rate_type = 'EPR_MERCHANT' then 'EPRM'
  when v2.rate_type = 'ESR_DIRECT_AGENCY' then 'ESRA'
  when v2.rate_type = 'EPR_DIRECT_AGENCY' then 'EPRA'
  when v2.rate_type = 'ESR_MERCHANT' then 'ESRM'
  else v2.rate_type end) as rate_type
,v2.hot_mip_rate
,collect_list(v3.amenity_description) as amnts
from eandev.HotelMessages
lateral view explode(travelers) travl_tbl as tuid
lateral view explode(hotels) hotel_tbl as v1
lateral view explode(v1.prices) rate_tbl as v2
lateral view explode(v2.rate_amenities) amt_tbl as v3
where load_date between '20200301' and '20200401'
and message_id is not null
and message_id <> ''
and v2.selected = True
and type = 'HSR'
group by message_id, message_timestamp, check_in_date, check_out_date, filter_rate_amenities,
filter_want_in_policy_rates_only, tuid, v1.hotel_id, v1.hotel_result_index,v1.sort_model_scores[0].main_score,v1.city, v1.country_code, v1.star_rating, v2.rate_index,v2.price.amount,v2.price.currency,
v2.rate_type,v2.hot_mip_rate
) as a
group by a.message_id, a.message_date, a.check_in_date, a.check_out_date,
a.filter_rate_amenities, a.filter_want_in_policy_rates_only, a.tuid, a.hotel_id,a.hotel_result_index,a.score, a.city, a.star_rating
'''

In [ ]:
'''
ssh Chwxedwhdc002.datawarehouse.expecn.com

hive -e "use eandev; INSERT OVERWRITE LOCAL DIRECTORY '/home/yixli/temp' ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' select * from yixli_search_commission_2019;"&

cd temp
ls | xargs -I '{}' -n1 sed -i 's/\\N//g' '{}' &
cd ..
cat temp/* >search_commission_2019.csv


~/.linuxbrew/bin/aws_key_gen login

aws s3 cp search_commission_2019.csv s3://ege-ds-workshops-corp/yixli/search_commission_2019.csv


'''

In [2]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                                  #1                
    T.StructField("message_date", T.TimestampType(), True),                                  #2                               #1                
    T.StructField("check_in_date", T.DateType(), True),                                 #3
    T.StructField("check_out_date", T.DateType(), True),                                #4
    T.StructField("tuid", T.IntegerType(), True),                                       #5
    T.StructField("hotel_id", T.IntegerType(), True),                                   #6 
     T.StructField("hotel_result_index", T.IntegerType(), True),                                   #6 
     T.StructField("city", T.StringType(), True),           #9                               
     T.StructField("star_rating", T.StringType(), True),           #9                               
    T.StructField("filter_want_in_policy_rates_only", T.BooleanType(), True),           #9    
    T.StructField("filter_eligible_for_loyalty", T.BooleanType(), True),                #10 
    T.StructField("filter_free_breakfast", T.BooleanType(), True),                      #11
    T.StructField("filter_free_wifi", T.BooleanType(), True),                           #12    
    T.StructField("filter_free_parking", T.BooleanType(), True),                        #13
    T.StructField("rate_index", T.StringType(), True),                                      #14
    T.StructField("rate_amount", T.StringType(), True),                                     #15
    T.StructField("rate_currency", T.StringType(), True),                                   #16
    T.StructField("rate_type", T.StringType(), True),                                       #17
    T.StructField("eligible_for_loyalty", T.StringType(), True),                            #20
    T.StructField("free_breakfast", T.StringType(), True),                                  #21
    T.StructField("free_wifi", T.StringType(), True),                                       #22
    T.StructField("free_parking", T.StringType(), True),                                    #23
    T.StructField("refundable", T.StringType(), True)                                       #24
])

In [3]:
file_loc = "s3://ege-ds-workshops-corp/yixli/src_amentity_2019.csv"
search_hotel_df = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191001' and '20200101'

search_hotel_df = search_hotel_df.\
            filter(F.col("hotel_id")>0).\
            filter(F.col('rate_amount').isNotNull())
print(search_hotel_df.count())

203379023

In [8]:
search_hotel_df.show(50)

+--------------------+--------------------+-------------+--------------+--------+--------+------------------+--------------------+-----------+--------------------------------+---------------------------+---------------------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          message_id|        message_date|check_in_date|check_out_date|    tuid|hotel_id|hotel_result_index|                city|star_rating|filter_want_in_policy_rates_only|filter_eligible_for_loyalty|filter_free_breakfast|filter_free_wifi|filter_free_parking|          rate_index|         rate_amount|       rate_currency|           rate_type|eligible_for_loyalty|      free_breakfast|           free_wifi|        free_parking|          refundable|
+--------------------+--------------------+-------------+--------------+--------+--------+------

In [9]:
#Oct
search_hotel_df1 = search_hotel_df.\
            filter(F.col("message_date") > F.unix_timestamp(F.lit('2019-10-01 00:00:00')).cast('timestamp')).\
            filter(F.col("message_date") < F.unix_timestamp(F.lit('2019-11-01 00:00:00')).cast('timestamp'))
print(search_hotel_df1.count())

86077534

In [ ]:
#Nov
search_hotel_df1 = search_hotel_df.\
            filter(F.col("message_date") > F.unix_timestamp(F.lit('2019-11-01 00:00:00')).cast('timestamp')).\
            filter(F.col("message_date") < F.unix_timestamp(F.lit('2019-12-01 00:00:00')).cast('timestamp'))
print(search_hotel_df1.count())

In [4]:
#Dec
search_hotel_df1 = search_hotel_df.\
            filter(F.col("message_date") > F.unix_timestamp(F.lit('2019-12-01 00:00:00')).cast('timestamp')).\
            filter(F.col("message_date") < F.unix_timestamp(F.lit('2020-1-01 00:00:00')).cast('timestamp'))
print(search_hotel_df1.count())

43621333

In [5]:
search_hotel_df1.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- check_in_date: date (nullable = true)
 |-- check_out_date: date (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- filter_want_in_policy_rates_only: boolean (nullable = true)
 |-- filter_eligible_for_loyalty: boolean (nullable = true)
 |-- filter_free_breakfast: boolean (nullable = true)
 |-- filter_free_wifi: boolean (nullable = true)
 |-- filter_free_parking: boolean (nullable = true)
 |-- rate_index: string (nullable = true)
 |-- rate_amount: string (nullable = true)
 |-- rate_currency: string (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- eligible_for_loyalty: string (nullable = true)
 |-- free_breakfast: string (nullable = true)
 |-- free_wifi: string (nullable = true)
 |-- free_parking: st

In [6]:
search_hotel_df1.show(50)

+--------------------+--------------------+-------------+--------------+--------+--------+------------------+--------------------+-----------+--------------------------------+---------------------------+---------------------+----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          message_id|        message_date|check_in_date|check_out_date|    tuid|hotel_id|hotel_result_index|                city|star_rating|filter_want_in_policy_rates_only|filter_eligible_for_loyalty|filter_free_breakfast|filter_free_wifi|filter_free_parking|          rate_index|         rate_amount|       rate_currency|           rate_type|eligible_for_loyalty|      free_breakfast|           free_wifi|        free_parking|          refundable|
+--------------------+--------------------+-------------+--------------+--------+--------+------

In [7]:
dir = 's3://ege-ds-workshops-corp/yixli/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_hotel_df1.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+'search_amentity_2019_12.csv',header = 'true')

In [8]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                                  #1                
    T.StructField("message_date", T.TimestampType(), True),                                  #2                               #1                
    T.StructField("check_in_date", T.DateType(), True),                                 #3
    T.StructField("check_out_date", T.DateType(), True),                                #4
    T.StructField("tuid", T.IntegerType(), True),                                       #5
    T.StructField("hotel_id", T.IntegerType(), True),                                   #6 
     T.StructField("hotel_result_index", T.IntegerType(), True),                                   #6 
     T.StructField("city", T.StringType(), True),           #9                               
     T.StructField("star_rating", T.StringType(), True),           #9                               
    T.StructField("filter_want_in_policy_rates_only", T.BooleanType(), True),           #9    
    T.StructField("filter_eligible_for_loyalty", T.BooleanType(), True),                #10 
    T.StructField("filter_free_breakfast", T.BooleanType(), True),                      #11
    T.StructField("filter_free_wifi", T.BooleanType(), True),                           #12    
    T.StructField("filter_free_parking", T.BooleanType(), True),                        #13
    T.StructField("rate_index", T.StringType(), True),                                      #14
    T.StructField("rate_amount", T.StringType(), True),                                     #15
    T.StructField("rate_currency", T.StringType(), True),                                   #16
    T.StructField("rate_type", T.StringType(), True),                                       #17
    T.StructField("eligible_for_loyalty", T.StringType(), True),                            #20
    T.StructField("free_breakfast", T.StringType(), True),                                  #21
    T.StructField("free_wifi", T.StringType(), True),                                       #22
    T.StructField("free_parking", T.StringType(), True),                                    #23
    T.StructField("refundable", T.StringType(), True)                                       #24
])

In [9]:
file_loc = "s3://ege-ds-workshops-corp/yixli/search_amentity_2019_12.csv"
print("Collecting search the data...")
search_hotel_df1 = sqlContext.read.format('csv').\
             options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)


search_hotel_df1 = search_hotel_df1.\
            filter(F.col("hotel_id")>0).\
            filter(F.col('rate_amount').isNotNull())
print(search_hotel_df1.count())

43621068

In [10]:
combine = F.udf(lambda x1,x2,x3,x4,x5,x6,x7,x8,x9: list(zip(x1,x2,x3,x4,x5,x6,x7,x8,x9)),             
              T.ArrayType(T.StructType([T.StructField("rate_index_a", T.StringType()),                                      #1
                                    T.StructField("rate_amount_a", T.StringType()),                                         #2
                                    T.StructField("rate_currency_a", T.StringType()),                                   #3
                                    T.StructField("rate_type_a", T.StringType()),                                       #4
                                   # T.StructField("hot_mip_rate_a", T.StringType()),                                       #4
                                    T.StructField("eligible_for_loyalty_a", T.StringType()),                            #7
                                    T.StructField("free_breakfast_a", T.StringType()),                                  #8
                                    T.StructField("free_wifi_a", T.StringType()),                                       #9
                                    T.StructField("free_parking_a", T.StringType()),                                    #10
                                    T.StructField("refundable_a", T.StringType()),                                    #10
                                    ])))

In [11]:
search_df1= search_hotel_df1.\
    withColumn("rate_index_a", F.split(F.col("rate_index"),"\|")).\
    withColumn("rate_amount_a", F.split(F.col("rate_amount"),"\|")).\
    withColumn("rate_currency_a", F.split(F.col("rate_currency"),"\|")).\
    withColumn("rate_type_a", F.split(F.col("rate_type"),"\|")).\
    withColumn("eligible_for_loyalty_a", F.split(F.col("eligible_for_loyalty"),"\|")).\
    withColumn("free_breakfast_a", F.split(F.col("free_breakfast"),"\|")).\
    withColumn("free_wifi_a", F.split(F.col("free_wifi"),"\|")).\
    withColumn("free_parking_a", F.split(F.col("free_parking"),"\|")).\
    withColumn("refundable_a", F.split(F.col("refundable"),"\|")).\
    withColumn("e", combine("rate_index_a", "rate_amount_a","rate_currency_a","rate_type_a",\
                "eligible_for_loyalty_a","free_breakfast_a","free_wifi_a","free_parking_a","refundable_a")).\
    withColumn("e",F.explode("e")).\
    select("message_id","message_date","check_in_date","check_out_date","hotel_id","hotel_result_index","tuid","city","star_rating", "filter_want_in_policy_rates_only",\
            "filter_eligible_for_loyalty", "filter_free_breakfast", "filter_free_wifi", "filter_free_parking",\
        F.col("e.rate_index_a").alias("rate_index"),\
        F.col("e.rate_amount_a").alias("rate_amount"),\
        F.col("e.rate_currency_a").alias("rate_currency"),\
        F.col("e.rate_type_a").alias("rate_type"),\
        F.col("e.eligible_for_loyalty_a").alias("eligible_for_loyalty"),\
        F.col("e.free_breakfast_a").alias("free_breakfast"),\
        F.col("e.free_wifi_a").alias("free_wifi"),\
        F.col("e.free_parking_a").alias("free_parking"),\
        F.col("e.refundable_a").alias("refundable"))

In [12]:
search_df1 = search_df1.\
            withColumn("rate_index", search_df1.rate_index.cast("integer")).\
            withColumn("rate_amount", search_df1.rate_amount.cast("float")).\
            withColumn("eligible_for_loyalty", search_df1.eligible_for_loyalty.cast("boolean")).\
            withColumn("free_breakfast", search_df1.free_breakfast.cast("boolean")).\
            withColumn("free_wifi", search_df1.free_wifi.cast("boolean")).\
            withColumn("free_parking", search_df1.free_parking.cast("boolean")).\
            withColumn("refundable", search_df1.refundable.cast("boolean"))

In [13]:
search_df1.count()

307581772

In [14]:
search_df1 = search_df1.filter(F.col('rate_type').isin('EPRA','EPRM','ESRM','ESRA','GDS','NEG'))

In [15]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_df1.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_amentity_2019_12_df.csv',header = 'true')

In [16]:
#file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-21-2020_search_amentity_2019_10_df.csv"
file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-24-2020_search_amentity_2019_12_df.csv"

print("Collecting search the data...")
search_df1 = sqlContext.read.format('csv').\
            options(header='true', inferSchema='true', delimiter=',').\
           load(file_loc)


search_df1 = search_df1.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("rate_amount").isNotNull())
print(search_df1.count())

307581772

In [17]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("hotel_id", T.IntegerType(), True),                   #6 
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("rate_index", T.StringType(), True),                 #8 
    T.StructField("commission_base", T.StringType(), True),              #12
    T.StructField("commission_tax", T.StringType(), True),               #14
    T.StructField("commission_currency", T.StringType(), True)          #16
])


In [18]:
file_loc = "s3://ege-ds-workshops-corp/yixli/search_commission_2019_v2.csv"
print("Collecting search the data...")
search_hotel_df = sqlContext.read.format('csv').\
            options(header='false', inferSchema='false', delimiter=',').\
            schema(search_schema).load(file_loc)

# load_date between '20191201' and '20200101'

search_hotel_df = search_hotel_df.\
            filter(F.col("hotel_id")>0)
print(search_hotel_df.count())

29960773

In [7]:
#Oct
search_df2 = search_hotel_df.\
            filter(F.col("message_date") > F.unix_timestamp(F.lit('2019-10-01 00:00:00')).cast('timestamp')).\
            filter(F.col("message_date") < F.unix_timestamp(F.lit('2019-11-01 00:00:00')).cast('timestamp'))
print(search_df2.count())

12686824

In [19]:
#Dec
search_df2 = search_hotel_df.\
            filter(F.col("message_date") > F.unix_timestamp(F.lit('2019-12-01 00:00:00')).cast('timestamp')).\
            filter(F.col("message_date") < F.unix_timestamp(F.lit('2020-01-01 00:00:00')).cast('timestamp'))
print(search_df2.count())

6419212

In [20]:
combine = F.udf(lambda x1,x2,x3,x4: list(zip(x1,x2,x3,x4)),             
              T.ArrayType(T.StructType([T.StructField("rate_index_a", T.StringType()),                                      #1
                                    T.StructField("commission_base_a", T.StringType()),                                         #2
                                    T.StructField("commission_tax_a", T.StringType()),                                   #3
                                    T.StructField("commission_currency_a", T.StringType())                                    
                                    ])))

In [21]:
search_df2= search_df2.\
    withColumn("rate_index_a", F.split(F.col("rate_index"),"\|")).\
    withColumn("commission_base_a", F.split(F.col("commission_base"),"\|")).\
    withColumn("commission_tax_a", F.split(F.col("commission_tax"),"\|")).\
    withColumn("commission_currency_a", F.split(F.col("commission_currency"),"\|")).\
    withColumn("e", combine("rate_index_a", "commission_base_a","commission_tax_a","commission_currency_a")).\
    withColumn("e",F.explode("e")).\
    select("message_id","check_in_date","check_out_date","hotel_id","tuid",\
        F.col("e.rate_index_a").alias("rate_index"),\
        F.col("e.commission_base_a").alias("commission_base"),\
        F.col("e.commission_tax_a").alias("commission_tax"),\
        F.col("e.commission_currency_a").alias("commission_currency"))

In [22]:
search_df2.count()

94785434

In [23]:
search_df2 = search_df2.\
            withColumn("rate_index", search_df2.rate_index.cast("integer")).\
            withColumn("commission_base", search_df2.commission_base.cast("float")).\
             withColumn("commission_tax", search_df2.commission_tax.cast("float"))
          

In [24]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_df2.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_commission_2019_12_df.csv',header = 'true')

In [25]:
#file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-21-2020_search_commission_2019_10_df.csv"

file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-24-2020_search_commission_2019_12_df.csv"

search_df2 = sqlContext.read.format('csv').\
            options(header='true', inferSchema='true', delimiter=',').\
           load(file_loc)

# load_date between '20191201' and '20200101'

search_df2 = search_df2.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("commission_base").isNotNull())
print(search_df2.count())

94785434

In [26]:
search_df = search_df1.\
             join(search_df2.select("commission_base","commission_tax","commission_currency","message_id","hotel_id","check_in_date", "check_out_date","tuid","rate_index"),
                  ["message_id","hotel_id","check_in_date", "check_out_date","tuid","rate_index"]).\
             dropDuplicates()

In [20]:
search_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

An error occurred while calling o344.showString.
: org.apache.spark.SparkException: Job 12 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:972)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:970)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:970)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2286)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2193)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.spark.scheduler.clus

In [27]:
search_df = search_df.withColumn("hotel_index", row_number().over(Window.partitionBy("message_id", "tuid").\
                                                        orderBy(F.asc("hotel_result_index"))))

In [28]:
search_df.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: timestamp (nullable = true)
 |-- check_out_date: timestamp (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- filter_want_in_policy_rates_only: boolean (nullable = true)
 |-- filter_eligible_for_loyalty: boolean (nullable = true)
 |-- filter_free_breakfast: boolean (nullable = true)
 |-- filter_free_wifi: boolean (nullable = true)
 |-- filter_free_parking: boolean (nullable = true)
 |-- rate_amount: double (nullable = true)
 |-- rate_currency: string (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- eligible_for_loyalty: boolean (nullable = true)
 |-- free_breakfast: boolean (nullable = true)
 |-- free_wifi: boolean (nullable = true)
 |-- fr

In [29]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_df.csv',header = 'true')

## Read search data

In [30]:
search_schema = T.StructType([
    T.StructField("message_id", T.StringType(), True),                  #1
    T.StructField("hotel_id", T.IntegerType(), True),                   #6
    T.StructField("check_in_date", T.DateType(), True),                 #3
    T.StructField("check_out_date", T.DateType(), True),                #4
    T.StructField("tuid", T.IntegerType(), True),                       #5
    T.StructField("rate_index", T.IntegerType(), True),                 #8 
    T.StructField("rate_type", T.StringType(), True),                   #11
    T.StructField("message_date", T.TimestampType(), True),             #2
    T.StructField("hotel_result_index", T.IntegerType(), True),         #7
    T.StructField("rate_amount", T.FloatType(), True),                  #9
    T.StructField("rate_currency", T.StringType(), True),               #10
    T.StructField("commission_base", T.FloatType(), True),              #12
    T.StructField("commission_tax", T.FloatType(), True),               #14
    T.StructField("commission_currency", T.StringType(), True),          #16
    T.StructField("hotel_index", T.IntegerType(), True)                 #8 
])


In [31]:
file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-24-2020_search_df.csv"
print("Collecting search the data...")
search_df = sqlContext.read.format('csv').\
            options(header='true', inferSchema='true', delimiter=',').\
            load(file_loc)


print(search_df.count())

62044274

In [32]:
search_df = search_df.\
            filter(F.col("hotel_id")>0).\
            filter(F.col("rate_amount").isNotNull())

In [33]:
search_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|          3756502|
|      NEG|          1175208|
|     EPRM|          7394933|
|     ESRM|         29364250|
|     ESRA|         13830871|
|      GDS|          6522510|
+---------+-----------------+

In [34]:
search_df.show()

+--------------------+--------+-------------------+-------------------+--------+----------+--------------------+------------------+------------------+-----------+--------------------------------+---------------------------+---------------------+----------------+-------------------+-----------+-------------+---------+--------------------+--------------+---------+------------+----------+---------------+--------------+-------------------+-----------+
|          message_id|hotel_id|      check_in_date|     check_out_date|    tuid|rate_index|        message_date|hotel_result_index|              city|star_rating|filter_want_in_policy_rates_only|filter_eligible_for_loyalty|filter_free_breakfast|filter_free_wifi|filter_free_parking|rate_amount|rate_currency|rate_type|eligible_for_loyalty|free_breakfast|free_wifi|free_parking|refundable|commission_base|commission_tax|commission_currency|hotel_index|
+--------------------+--------+-------------------+-------------------+--------+----------+-----

In [35]:
search_df.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: timestamp (nullable = true)
 |-- check_out_date: timestamp (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- filter_want_in_policy_rates_only: boolean (nullable = true)
 |-- filter_eligible_for_loyalty: boolean (nullable = true)
 |-- filter_free_breakfast: boolean (nullable = true)
 |-- filter_free_wifi: boolean (nullable = true)
 |-- filter_free_parking: boolean (nullable = true)
 |-- rate_amount: double (nullable = true)
 |-- rate_currency: string (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- eligible_for_loyalty: boolean (nullable = true)
 |-- free_breakfast: boolean (nullable = true)
 |-- free_wifi: boolean (nullable = true)
 |-- fr

## Filter hotels below peterman_line

In [10]:
search_df.registerTempTable("search_df")
peterman_df.registerTempTable("peterman_df")

In [11]:
search_below_peterman_df = sqlContext.sql("select\
                             b.*\
                             from search_df as b join peterman_df as c on (b.message_id=c.message_id)\
                             where (b.hotel_result_index>c.last_idx_peterman_line)\
                             order by b.message_id, b.hotel_result_index").\
                             dropDuplicates()

In [12]:
search_below_peterman_df.filter(F.col('rate_type')=='NEG').count()

0

In [36]:
search_below_peterman_df = search_df.filter(F.col('rate_type')!='NEG')

In [37]:
search_below_peterman_df.count()

60869066

In [39]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_below_peterman_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_below_peterman_df.csv',header = 'true')

In [2]:
file_loc = "s3://ege-ds-workshops-corp/yixli/data_understanding/09-24-2020_search_below_peterman_df.csv"
print("Collecting search the data...")
search_below_peterman_df = sqlContext.read.format('csv').\
            options(header='true', inferSchema='true', delimiter=',').\
            load(file_loc)

print(search_below_peterman_df.count())

60869066

## Exchange currency to USD

In [3]:
sqlCurrency = """select DISTINCT
                 a.exchangeratedate as timestamp,
                 trunc(a.exchangeratedate) as exch_rate_date,
                 a.fromcurrencycode,
                 a.exchangerate
                 from public.exchangeratedailyfull a
                 where trunc(a.exchangeratedate)>=to_date('20191201','YYYYMMDD') 
                 and trunc(a.exchangeratedate)<=to_date('20200101','YYYYMMDD') 
                 and tocurrencycode = 'USD' """
connection_string = "jdbc:redshift://egencia-reporting.czjkedodj6lc.us-west-2.redshift.amazonaws.com:5439/egedatamart?user=ds_rpt_user&password=Jan2019!";

currency_df = sqlContext.read.\
    format("com.databricks.spark.redshift").\
    option("url", connection_string).\
    option("query", sqlCurrency).\
    option("tempdir", "s3a://ege-ds-workshops-corp/yixli/").\
    load().\
    dropDuplicates().\
    orderBy(["exch_rate_date","fromcurrencycode"],ascending=True)

In [5]:
local_below_peterman_df = search_below_peterman_df.filter(F.col('rate_type')!='GDS').filter(F.col('commission_base').isNotNull())
local_below_peterman_df.registerTempTable("local_below_peterman_df")
currency_df.registerTempTable("currency_df")

In [6]:
local_below_peterman_usd_df = sqlContext.sql("select\
                             a.*,\
                             a.rate_amount*COALESCE(ex1.exchangerate,1) AS src_rate_amount_usd,\
                             a.commission_base*COALESCE(ex2.exchangerate,1) AS src_commission_base_usd,\
                             a.commission_tax*COALESCE(ex2.exchangerate,1) AS src_commission_tax_usd\
                             from local_below_peterman_df a\
                             left join currency_df ex1 on to_date(a.message_date)=ex1.exch_rate_date and a.rate_currency=ex1.FromCurrencyCode\
                             left join currency_df ex2 on to_date(a.message_date)=ex2.exch_rate_date and a.commission_currency=ex2.FromCurrencyCode").\
                             dropDuplicates()

In [7]:
local_below_peterman_usd_df = local_below_peterman_usd_df.\
withColumn('src_supply_revenue_usd',F.col('src_commission_base_usd')+F.col('src_commission_tax_usd'))

In [8]:
local_below_peterman_usd_df = local_below_peterman_usd_df.filter(F.col('src_supply_revenue_usd')<1000)

In [9]:
local_below_peterman_usd_df.groupBy('rate_type').agg(F.count(F.col('message_id'))).show()

+---------+-----------------+
|rate_type|count(message_id)|
+---------+-----------------+
|     EPRA|          3756490|
|     EPRM|          7394654|
|     ESRM|         29361273|
|     ESRA|         13830426|
+---------+-----------------+

In [10]:
sqlGDS =  """SELECT 
      a.hotelid as hotel_id,
     trunc(a.issuedate) as issue_date,
     trunc(a.traveldatestart) as check_in_date,
     trunc(a.traveldateend) as check_out_date,
      (case when a.customersystemid=1 then a.TUIDTraveler else g.TUIDInternal end) as tuid,      
      a.travelproductid AS travel_product_id,
      a.GroupAccountID AS group_account_id,
      hsp.rate_type as hotel_rate_type,
      (case  
        when a.hotelrateTypeSupplyid IN (1,2,3,4,5,10,11,14,16,17,18,20,25,26,28,29,30,32) then 'GDS'
        when a.hotelrateTypeSupplyid IN (12,13,15,19,21,22,23,27,31,33,34) then 'NEG'
        when a.hotelrateTypeSupplyid = 6 then 'ESRA'
        when a.hotelrateTypeSupplyid = 7 then 'ESRM'
        when a.hotelrateTypeSupplyid = 8 then 'EPRA'
        when a.hotelrateTypeSupplyid = 9 then 'EPRM'
        when a.hotelrateTypeSupplyid IN (24,35) then 'HOTMIP'
        else 'UNK'
        end) as bk_rate_type,
      NVL(hsp.commission,0.00) As commission_rate,
      4.30 AS gds
FROM public.HotelFact a
  JOIN TravelerAccountDim g 
    ON a.TUIDTraveler = g.TUID AND a.customersystemid=g.customersystemid 
  JOIN public.GroupAccountdim d 
    ON a.GroupAccountID = d.GroupAccountID 
  JOIN public.hotel_supply hsp 
    ON hsp.year = EXTRACT (year FROM a.IssueDate)
    AND hsp.quarter = EXTRACT (quarter FROM a.IssueDate)
    AND hsp.rate_type = CASE
         WHEN a.HotelRateTypeSupplyID IN (7,9,24) THEN 'Expedia Collect'
         WHEN a.HotelRateTypeSupplyID IN (6,8,35) THEN 'Hotel Collect'
         WHEN a.HotelRateTypeSupplyID IN (12,13,15,19,21,22,23,27,31,33,34) THEN 'Negotiated'
         ELSE 'Published & GDS' END
    AND hsp.travel_product_id = a.TravelProductID
WHERE a.CustomerSystemID IN (1,2)
AND   (d.groupaccountinternaltypeid = 1 OR d.groupaccountinternaltypeid = 3)
AND a.BookingTypeID in (1,3) 
and hsp.year=2019
and hsp.quarter=4""".replace('\n',' ')
            
connection_string = "jdbc:redshift://egencia-reporting.czjkedodj6lc.us-west-2.redshift.amazonaws.com:5439/egedatamart?user=ds_rpt_user&password=Jan2019!";

gds_df = sqlContext.read.\
    format("com.databricks.spark.redshift").\
    option("url", connection_string).\
    option("query", sqlGDS).\
    option("tempdir", "s3a://ege-ds-workshops-corp/yixli/").\
    load().\
    filter(F.col("hotel_rate_type")=='Published & GDS').\
    filter(F.col('commission_rate').isNotNull()).\
    filter(F.col('gds').isNotNull()).\
    dropDuplicates()

In [11]:
gds_df = gds_df.groupBy('hotel_id').agg(F.avg('commission_rate').alias('commission_rate'),
                                       F.avg('gds').alias('gds'))

In [12]:
gds_df.show(50)

+--------+--------------------+--------+
|hotel_id|     commission_rate|     gds|
+--------+--------------------+--------+
|32146635|0.080000000000000...|4.300000|
|  904712|0.080000000000000...|4.300000|
|  517191|0.080000000000000...|4.300000|
|   63964|0.080000000000000...|4.300000|
| 1601034|0.080000000000000...|4.300000|
|11785356|0.080000000000000...|4.300000|
|  525571|0.080000000000000...|4.300000|
|  798779|0.080000000000000...|4.300000|
|  851284|0.080000000000000...|4.300000|
|18629649|0.080000000000000...|4.300000|
| 5058675|0.080000000000000...|4.300000|
|17347478|0.080000000000000...|4.300000|
|33374019|0.080000000000000...|4.300000|
|  904455|0.080000000000000...|4.300000|
|   17753|0.080000000000000...|4.300000|
|  802387|0.080000000000000...|4.300000|
|23814718|0.080000000000000...|4.300000|
|11484251|0.080000000000000...|4.300000|
|13125518|0.080000000000000...|4.300000|
|    3918|0.080000000000000...|4.300000|
| 1636908|0.080000000000000...|4.300000|
|  968049|0.0800

In [13]:
gds_below_peterman_df = search_below_peterman_df.filter(F.col('rate_type')=='GDS')
gds_below_peterman_df = gds_below_peterman_df.join(currency_df,
                                                                [F.to_date(gds_below_peterman_df.message_date)==currency_df.exch_rate_date,
                                                                gds_below_peterman_df.rate_currency==currency_df.fromcurrencycode]).\
                        withColumn('src_rate_amount_usd',gds_below_peterman_df.rate_amount*currency_df.exchangerate)                                             
gds_below_peterman_df.count()

6522510

In [14]:
gds_below_peterman_df = gds_below_peterman_df.\
join(gds_df.select('hotel_id','gds','commission_rate'),['hotel_id'],how='left').\
withColumn('src_supply_revenue_usd',F.col('src_rate_amount_usd')*F.col('commission_rate')+F.col('gds'))

In [15]:
gds_below_peterman_df = gds_below_peterman_df.filter(F.col('src_supply_revenue_usd')<1000)

In [16]:
gds_below_peterman_df.count()

5523118

In [17]:
gds_below_peterman_df = gds_below_peterman_df.withColumn('src_commission_base_usd',F.lit(0.0))

In [18]:
local_below_peterman_usd_df.columns

['message_id', 'hotel_id', 'check_in_date', 'check_out_date', 'tuid', 'rate_index', 'message_date', 'hotel_result_index', 'city', 'star_rating', 'filter_want_in_policy_rates_only', 'filter_eligible_for_loyalty', 'filter_free_breakfast', 'filter_free_wifi', 'filter_free_parking', 'rate_amount', 'rate_currency', 'rate_type', 'eligible_for_loyalty', 'free_breakfast', 'free_wifi', 'free_parking', 'refundable', 'commission_base', 'commission_tax', 'commission_currency', 'hotel_index', 'src_rate_amount_usd', 'src_commission_base_usd', 'src_commission_tax_usd', 'src_supply_revenue_usd']

In [19]:
search_revenue_below_peterman_usd_df = local_below_peterman_usd_df.select('message_id', 'hotel_id', 'check_in_date', 'check_out_date', 'tuid', 'rate_index', 'message_date', 'hotel_result_index', 'city', 'star_rating', 'filter_want_in_policy_rates_only', 'filter_eligible_for_loyalty', 'filter_free_breakfast', 'filter_free_wifi', 'filter_free_parking',  'rate_type', 'eligible_for_loyalty', 'free_breakfast', 'free_wifi', 'free_parking', 'refundable',  'src_rate_amount_usd', 'src_commission_base_usd', 'src_supply_revenue_usd').\
                            union(gds_below_peterman_df.select('message_id', 'hotel_id', 'check_in_date', 'check_out_date', 'tuid', 'rate_index', 'message_date', 'hotel_result_index', 'city', 'star_rating', 'filter_want_in_policy_rates_only', 'filter_eligible_for_loyalty', 'filter_free_breakfast', 'filter_free_wifi', 'filter_free_parking',  'rate_type', 'eligible_for_loyalty', 'free_breakfast', 'free_wifi', 'free_parking', 'refundable',  'src_rate_amount_usd', 'src_commission_base_usd', 'src_supply_revenue_usd'))

In [20]:
search_revenue_below_peterman_usd_df.count()

59865961

In [21]:
search_revenue_below_peterman_usd_df.groupBy('rate_type').agg(F.avg('src_supply_revenue_usd')).show()

+---------+---------------------------+
|rate_type|avg(src_supply_revenue_usd)|
+---------+---------------------------+
|     EPRA|          27.91742984711789|
|     EPRM|         37.865968243569846|
|     ESRM|         34.585569360666746|
|     ESRA|          25.54032326778975|
|      GDS|          19.49274424419558|
+---------+---------------------------+

In [22]:
dir = 's3://ege-ds-workshops-corp/yixli/data_understanding/'

datestamp = datetime.datetime.now().strftime('%m-%d-%Y')
search_revenue_below_peterman_usd_df.repartition(1).write.format('com.databricks.spark.csv').mode('overwrite')\
.save(dir+datestamp+'_search_revenue_below_peterman_usd_df.csv',header = 'true')

In [23]:
search_revenue_below_peterman_usd_df.printSchema()

root
 |-- message_id: string (nullable = true)
 |-- hotel_id: integer (nullable = true)
 |-- check_in_date: timestamp (nullable = true)
 |-- check_out_date: timestamp (nullable = true)
 |-- tuid: integer (nullable = true)
 |-- rate_index: integer (nullable = true)
 |-- message_date: timestamp (nullable = true)
 |-- hotel_result_index: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- filter_want_in_policy_rates_only: boolean (nullable = true)
 |-- filter_eligible_for_loyalty: boolean (nullable = true)
 |-- filter_free_breakfast: boolean (nullable = true)
 |-- filter_free_wifi: boolean (nullable = true)
 |-- filter_free_parking: boolean (nullable = true)
 |-- rate_type: string (nullable = true)
 |-- eligible_for_loyalty: boolean (nullable = true)
 |-- free_breakfast: boolean (nullable = true)
 |-- free_wifi: boolean (nullable = true)
 |-- free_parking: boolean (nullable = true)
 |-- refundable: boolean (nullable = true)
 |-- sr